In [1]:
import pymongo
import json
import csv
import pandas as pd
import timeit

In [ ]:
client = pymongo.MongoClient()
db = client['fukushima'] #make database

#MEXT Data

In [2]:

mcol = db['mext'] #make collection for mext
mcol.remove()

chunks = pd.read_csv('original_data/mext_gammadoseratereadingsbyprefecture.csv',chunksize=50000, usecols=['Time Date JST', 'Dose Rate',
       'Latitude', 'Longitude','Comments'])
f = open('mext_data.csv','w')
start = timeit.default_timer()
i = 0
for c in chunks:
    c = c[c.Comments.isnull()] #remove malfunctioning data
    c = c.drop('Comments',axis=1)
    c['Dose Rate'] = c['Dose Rate'].astype('float64')
    c['Time Date JST'] = c['Time Date JST'].astype('datetime64[ns]')
    c.columns = ['datetime','doserate','lat','long']
    data_json = json.loads(c.to_json(orient='records',date_format='iso'))
    mcol.insert(data_json)
    c.to_csv(f,index=False)
    i += 1
f.close()
elapsed = timeit.default_timer()-start
print('Processed ~{:.3e} lines ({} chunks) in {:.2f} sec'.format(i*50000,i,elapsed))
print('Approximately {:.2e} lines/sec'.format(i*50000/elapsed))
    
    

Processed ~3.500e+05 lines (7 chunks) in 12.38 sec
Approximately 2.83e+04 lines/sec


#Safecast Data

In [3]:

# device_id  uSvh=CPM/x      Reciprocal Estimate
# -- =================  ==========     ====================
# --              NULL       350.0       0.0028571428571429
# --5,15,16,17,18,22,69,89   350.0       0.0028571428571429
# --      6,7,11,13,23       100.0                     0.01
# --   4,9,10,12,19,24       132.0       0.0075757575757576
# --                21      1750.0    0.0005714285714285714
# --  21 & user_id=530     11000.0    0.0000909090909090909  *** TEMPORARY ***
def convert_cpm(row):
    """This function is based on the above table taken from safecast's publicly available code to
    convert cpm to uSv/h. MEXT doserates are already in uSv/h"""
    conversion_dict = {
        5: 0.0028571428571429,
        15: 0.0028571428571429,
        16: 0.0028571428571429,
        17: 0.0028571428571429,
        18: 0.0028571428571429,
        22: 0.0028571428571429,
        69: 0.0028571428571429,
        89: 0.0028571428571429,
        6: 0.01,
        7: 0.01,
        11: 0.01,
        13: 0.01,
        23: 0.01,
        4:0.0075757575757576,
        9:0.0075757575757576,
        10:0.0075757575757576,
        12:0.0075757575757576,
        19:0.0075757575757576,
        24:0.0075757575757576,
        21:0.0005714285714285714}
    did = row['Device ID']
    try:
        if pd.isnull(did):
#             print('nan')
            m = 0.0028571428571429
            return row['Value']*m
        else:
            m = conversion_dict[did]
            if row['Value'] == 'cpm':
                return row['Value']*m
            else:
                raise KeyError
    except KeyError:
#         print('keyerr')
        return np.nan
        

In [ ]:
scol = db['safecast'] #make collection for safecast
# scol.remove()
chunks = pd.read_csv('original_data/mclean.csv',chunksize=50000, usecols=['User ID', 'Captured Time', 'Latitude', 'Longitude', 'Value',
       'Unit', 'Device ID'])
i = 0
f = open('safecast_data.csv','w')
start = timeit.default_timer()
for c in chunks:
    c['doserate'] = c.apply(convert_cpm, axis=1)
    c['datetime'] = c['Captured Time'].astype('datetime64[ns]')
    c.drop(['Captured Time','Value','Unit'], axis=1, inplace=True)
    c.rename(columns={'Latitude': 'lat','Longitude': 'long'}, inplace=True)
    data_json = json.loads(c.to_json(orient='records',date_format='iso'))
    scol.insert(data_json)
    c.to_csv(f,index=False)
#     if c['Device ID'].any():
#         print(c['Device ID'].value_counts())
#         print(i)
#         break
    
    
    i += 1
    if i % 100 == 0:
        print(i)
#     if i ==20:
#         break
f.close()
elapsed = timeit.default_timer()-start
print('Processed ~{:.3e} lines ({} chunks) in {:.2f} sec'.format(i*50000,i,elapsed))
print('Approximately {:.2e} lines/sec'.format(i*50000/elapsed))

Collection(Database(MongoClient('localhost', 27017), 'fukushima'), 'safecast')

In [ ]:
#example query
for i in mcol.find({'Dose Rate': {'$gt': 2.2}}):
    print(i)